In [1]:
import pandas as pd
import numpy as np

# Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [2]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [3]:
print('Does the wine df conatin nulls?:', wine.isnull().any().any())
print('Does the energy df contain nulls?:', energy.isnull().any().any())

Does the wine df conatin nulls?: True
Does the energy df contain nulls?: False


In [4]:
wine.isnull().any().any()

True

**Question:** why did we use `.any()` twice when asking this question?

What happens if we just use `.any()` once?

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

In [5]:
print('Shape of wine table before is: ', wine.shape)

null_country = wine.loc[wine['country'].isnull()]
print('Shape of null countries table is:', null_country.shape)

wine = wine.loc[~wine.index.isin(null_country.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150930, 10)
Shape of null countries table is: (5, 10)
Shape of wine table after is:  (150925, 10)


The 5 rows where a NaN value existed for country have been dropped.

Now we'll do the same for rows with a null price.

In [6]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine.loc[wine['price'].isnull()]
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150925, 10)
Shape of null prices table is: (13695, 10)
Shape of wine table after is:  (137230, 10)


Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

In [7]:
wine.isnull().any()

country        False
description    False
designation     True
points         False
price          False
province       False
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

In [8]:
wine.fillna('unknown', inplace=True)

We now have no null values in our dataset!

In [9]:
wine.isnull().any()

country        False
description    False
designation    False
points         False
price          False
province       False
region_1       False
region_2       False
variety        False
winery         False
dtype: bool

In [10]:
# visualizing the df to see rows where "unknown" has been inserted
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude


### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

In [11]:
wine.loc[:, 'Region'] = wine.loc[:, 'region_1'] + '_' + wine.loc[:, 'region_2']

#### Using a function + apply to make a new column:
Let's say we want to make a new column with an ID we can use to identify the records.
We can use a helper function + the apply method in pandas to apply it to all rows.
Apply looks through each row or column of a dataframe (depending on Axis set).

In [12]:
def add_unique_id(row):
    row['New_ID'] = (row['country'] + "_" + 
                        row['winery'] + "_" + 
                        str(np.random.randint(low=1, high=1000)))
    return row

In [13]:
wine = wine.apply(add_unique_id, axis=1)

In [14]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Region,New_ID
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Napa Valley_Napa,US_Heitz_177
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez,Toro_unknown,Spain_Bodega Carmen Rodríguez_280
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,Knights Valley_Sonoma,US_Macauley_957
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,Willamette Valley_Willamette Valley,US_Ponzi_63
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude,Bandol_unknown,France_Domaine de la Bégude_989


We can use the same approach to make a column for "great and cheap" wines, defined here as $50 or less and with a score >95.

In [15]:
def helper(row):
    if (row['price'] < 50) & (row['points'] >=95):
        x = 1
    else:
        x = 0
    return x

In [16]:
wine.price = wine.price.astype('int')

In [17]:
wine.loc[:, 'great and cheap'] = wine.apply(helper, axis=1)

We can then use that boolean value to sort on the wines.

In [18]:
wine.loc[wine.loc[:, 'great and cheap']==1]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Region,New_ID,great and cheap
11,US,"From 18-year-old vines, this supple well-balan...",Estate Vineyard Wadensvil Block,95,48,Oregon,Ribbon Ridge,Willamette Valley,Pinot Noir,Patricia Green Cellars,Ribbon Ridge_Willamette Valley,US_Patricia Green Cellars_70,1
12,US,A standout even in this terrific lineup of 201...,Weber Vineyard,95,48,Oregon,Dundee Hills,Willamette Valley,Pinot Noir,Patricia Green Cellars,Dundee Hills_Willamette Valley,US_Patricia Green Cellars_790,1
20,US,Heitz has made this stellar rosé from the rare...,Grignolino,95,24,California,Napa Valley,Napa,Rosé,Heitz,Napa Valley_Napa,US_Heitz_267,1
24,US,"San Jose-based producer Adam Comartin heads 1,...",R-Bar-R Ranch,95,45,California,Santa Cruz Mountains,Central Coast,Pinot Noir,Comartin,Santa Cruz Mountains_Central Coast,US_Comartin_399,1
2439,US,This is possibly the most savory rendition of ...,Estate,96,40,California,San Luis Obispo County,Central Coast,Syrah,Stolo,San Luis Obispo County_Central Coast,US_Stolo_888,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143542,US,"What a spectacular effort. Dense and precise, ...",unknown,95,40,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Le Cadeau,Willamette Valley_Willamette Valley,US_Le Cadeau_800,1
143594,US,"Firm, chewy, young, tight and yet quite supple...",Père de Famille,95,45,Washington,Columbia Valley (WA),Columbia Valley,Cabernet Sauvignon,Betz Family,Columbia Valley (WA)_Columbia Valley,US_Betz Family_844,1
143684,France,"This Champagne, full of richness and fresh aci...",Cuvée 729,95,45,Champagne,Champagne,unknown,Champagne Blend,Jacquesson et Fils,Champagne_unknown,France_Jacquesson et Fils_247,1
145140,US,A fabulous white Rhone blend of enormous compl...,Esprit de Beaucastel Blanc,95,35,California,Paso Robles,Central Coast,White Blend,Tablas Creek,Paso Robles_Central Coast,US_Tablas Creek_375,1


**Exercise:** Make a new column in the energy dataset, showing the energy in time T-1 for each time T. Hint: there is a built-in method in Pandas that can help with this, no complicated functions needed :)

In [19]:
energy.loc[:, 'energy_t-1'] = energy.loc[:, 'PJM_Load_MW'].shift()

In [20]:
energy.head()

,PJM_Load_MW,energy_t-1
Datetime,,
1998-12-31 01:00:00,29309.0,NaN
1998-12-31 02:00:00,28236.0,29309.0
1998-12-31 03:00:00,27692.0,28236.0
1998-12-31 04:00:00,27596.0,27692.0
1998-12-31 05:00:00,27888.0,27596.0


**Exercise:** Re-read the CSV for the wine data, and write a function to take care of all the nan values. Bonus points if it includes a test.

In [21]:
wine1 = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)

def fix_null_values(wine_df, cols_to_drop, cols_to_fill):
    """
    Function to remove null values in the wine dataframe.
    Args:
        wine_df (DataFrame): DataFrame from which to remove the null values.
        cols_to_drop (list): List of column names to drop rows from
        cols_to_fill (list): List of columns names to fill with "unknown"
    Returns:
        wine_df (DataFrame): DataFrame with null values removed
    """
    for col in cols_to_drop:
        indicies_to_drop = wine_df.loc[wine_df[col].isnull()]
        wine_df = wine_df[~wine_df.index.isin(indicies_to_drop.index)]
        
    for col in cols_to_fill:
        wine_df.loc[:, col] = "unknown"
        
    # checking that all null values have been removed
    if wine_df.isnull().any().any() == False:
        return wine_df
    
    # if there are still null values, returning column name to user
    else:
        all_cols_list = list(wine_df.isnull().any())
        list_cols_with_nulls = [col for i, col in enumerate(all_cols_list.index) if all_cols_list[i] is True]
        print(f"Null values still exist! Columns {list_cols_with_nulls}")
        return
        
to_drop = ["price", "country"]
to_fill = ['description', 'designation', 'points', 'province', 'region_1', 'region_2', 'variety', 'winery']

# calling funciton
fix_null_values(wine1, to_drop, to_fill)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,unknown,unknown,unknown,235.0,unknown,unknown,unknown,unknown,unknown
1,Spain,unknown,unknown,unknown,110.0,unknown,unknown,unknown,unknown,unknown
2,US,unknown,unknown,unknown,90.0,unknown,unknown,unknown,unknown,unknown
3,US,unknown,unknown,unknown,65.0,unknown,unknown,unknown,unknown,unknown
4,France,unknown,unknown,unknown,66.0,unknown,unknown,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...
150925,Italy,unknown,unknown,unknown,20.0,unknown,unknown,unknown,unknown,unknown
150926,France,unknown,unknown,unknown,27.0,unknown,unknown,unknown,unknown,unknown
150927,Italy,unknown,unknown,unknown,20.0,unknown,unknown,unknown,unknown,unknown
150928,France,unknown,unknown,unknown,52.0,unknown,unknown,unknown,unknown,unknown


## Updates: Pandas 1.0
Pandas had already been a well-tested package used in development environments for quite some time. The official move to version 1.0 mainly just included some format fixes and 
- Dedicated types for String, boolean and NaN (before, just called "object")
- New documentation


In [22]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [23]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Region,New_ID,great and cheap
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Napa Valley_Napa,US_Heitz_177,0
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez,Toro_unknown,Spain_Bodega Carmen Rodríguez_280,0
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,Knights Valley_Sonoma,US_Macauley_957,0
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,Willamette Valley_Willamette Valley,US_Ponzi_63,0
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude,Bandol_unknown,France_Domaine de la Bégude_989,0


In [24]:
wine.convert_dtypes()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Region,New_ID,great and cheap
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Napa Valley_Napa,US_Heitz_177,0
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez,Toro_unknown,Spain_Bodega Carmen Rodríguez_280,0
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,Knights Valley_Sonoma,US_Macauley_957,0
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,Willamette Valley_Willamette Valley,US_Ponzi_63,0
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude,Bandol_unknown,France_Domaine de la Bégude_989,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150925,Italy,Many people feel Fiano represents southern Ita...,unknown,91,20,Southern Italy,Fiano di Avellino,unknown,White Blend,Feudi di San Gregorio,Fiano di Avellino_unknown,Italy_Feudi di San Gregorio_312,0
150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27,Champagne,Champagne,unknown,Champagne Blend,H.Germain,Champagne_unknown,France_H.Germain_791,0
150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20,Southern Italy,Fiano di Avellino,unknown,White Blend,Terredora,Fiano di Avellino_unknown,Italy_Terredora_296,0
150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52,Champagne,Champagne,unknown,Champagne Blend,Gosset,Champagne_unknown,France_Gosset_511,0
